In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, SeleniumURLLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
import chromadb

In [3]:
website = "https://hackernoon.com/vector-databases-getting-started-with-chromadb-and-more"

def load_document(loader_class, website_url):
    loader = loader_class([website_url])
    return loader.load()

wb_loader_doc = load_document(WebBaseLoader, website)
wb_loader_doc[0].page_content


"Vector Databases: Getting Started With ChromaDB and More | HackerNoonReadWriteNotificationssee  moreLOGIN / SIGNUPVector Databases: Getting Started With ChromaDB and More‚ÄÇby@shyamganesh 3,100 readsVector Databases: Getting Started With ChromaDB and MoreFebruary 13th 20245m by  @shyamganesh  3,100 readsToo Long; Didn't ReadIn this article, we will explore another well-known vector store called ChromaDB. Chroma DB is a vector store that is open-source and is utilized for the retrieval of vector embeddings. We will create a vectors store, add collections, add text to the collection, and perform a query search with and without meta filtering.cloud #vector-database #vector-search@shyamganeshShyam Ganesh SAn IT professional with a keen interest in delving into the realm of Machine Learning.Receive Stories from @shyamganeshSUBSCRIBE SUBSCRIBE TO RECEIVE THIS WRITER'S CONTENT STRAIGHT TO YOUR INBOX!Credibilityby Shyam Ganesh S @shyamganesh.An IT professional with a keen interest in delving 

In [4]:
selenium_loader_doc = load_document(SeleniumURLLoader, website)
selenium_loader_doc[0].page_content

'3,100 reads\n\nVector Databases: Getting Started With ChromaDB and More\n\nFebruary 13th 2024\n\n5min\n\nby  @shyamganesh\n\nShyam Ganesh S\n\n@shyamganesh\n\nAn IT professional with a keen interest in delving into...\n\n3,100 reads\n\nRead this story in a terminal\n\nPrint this story\n\nRead this story w/o Javascript\n\nToo Long; Didn\'t ReadIn this article, we will explore another well-known vector store called ChromaDB. Chroma DB is a vector store that is open-source and is utilized for the retrieval of vector embeddings. We will create a vectors store, add collections, add text to the collection, and perform a query search with and without meta filtering.\n\ncloud\n\n#vector-database\n\n#vector-search\n\n#vector-embedding\n\n#langchain\n\n#machine-learning\n\n#machinelearning\n\n#nlp\n\n#vector\n\n1x\n\nRead by Dr. One (en-US)\n\nAudio Presented by\n\n@shyamganesh\n\nAn IT professional with a keen interest in delving into the realm of Machine Learning.\n\nReceive Stories from @shy

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=200)
splits = text_splitter.split_documents(selenium_loader_doc)
splits[2]

Document(page_content='February 13th 2024\n\n5min\n\nby  @shyamganesh\n\nShyam Ganesh S\n\n@shyamganesh\n\nAn IT professional with a keen interest in delving into...\n\n3,100 reads\n\nRead this story in a terminal\n\nPrint this story', metadata={'source': 'https://hackernoon.com/vector-databases-getting-started-with-chromadb-and-more', 'title': 'Vector Databases: Getting Started With ChromaDB and More | HackerNoon', 'description': 'In this article, we will explore another well-known vector store called ChromaDB. Chroma DB is a vector store that is open-source.', 'language': 'en'})

In [6]:
import chromadb
import chromadb.utils.embedding_functions as embedding_functions
#client = chromadb.PersistentClient(path="/src/database/chroma_data")

#client = chromadb.HttpClient(host="localhost", port=8000)
#collection = client.create_collection("open2_collection", embedding_function=embedding_functions.OpenAIEmbeddingFunction(api_key=os.environ["OPENAI_API_KEY"]))

In [7]:
#collection = client.get_collection("open2_collection")
#for n in range(0, 10):
#   collection.add(documents=[splits[n].page_content], ids=['id' + str(n)])

In [8]:
#You are an assistant for helping refugees to get settled in Germany and give them individual guidenace. Use the following pieces of retrieved context to answer the question and ask follow-up questions. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Make sure to give a personal experience by asking questions about the personal situation to provide better answers for the next question.

prompt = PromptTemplate.from_template(
    """
    This is just a test prompt.
    Question: {question} 
    Context: {context} 
    Answer:
    """
)

In [9]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [10]:
vectorstore = Chroma.from_documents(splits, embedding=OpenAIEmbeddings())
#vectorstore.persist()
retriever = vectorstore.as_retriever()

In [11]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

#embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [12]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [13]:
response = rag_chain.invoke("What is the context you are getting?")
response

'The context I am getting is related to vector databases, specifically ChromaDB. The output displays documents that have similarities to the query document "University of Architecture" and includes document IDs, distances, and metadata of these documents. The metadata includes information about ChromaDB being an open-source vector store. The source of this information is a HackerNoon article titled "Vector Databases: Getting Started With ChromaDB and More".'